In [89]:
import pynucastro as pyna
reaclibrary = pyna.ReacLibLibrary()
all_nuclei = ["p","n", "h2", "h3", "he3", "he4","Li6","Li7","Be7","Li8","B8","Be9","B10","B11","C11","B12","C12","N12","C13","N13","C14","N14","O14","N15","O15","O16"]
bbn_library = reaclibrary.linking_nuclei(all_nuclei)
bbn_network = pyna.networks.PythonNetwork(libraries=bbn_library)


In [90]:
p__n_string = '''
b0 = -0.62173 
b1 = 0.22211e2
b2 = -0.72798e2
b3 = 0.11571e3
b4 = -0.11763e2
b5 = 0.45521e2
b6 = -3.7973 
b7 = 0.41266 
b8 = -0.026210
b9 = 0.87934e-3
b10 = -0.12016e-4
qpn = 2.8602

@numba.njit() 
def p__n(rate_eval, tf):  
    # p --> n
    z=5.92989658*tf.T9i
    rate=0
    #rate from https://arxiv.org/pdf/astro-ph/0408076.pdf appendix C
    b=[b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10]
    if tf.T9>1.160451812:
      for i in range(11):
         rate+=1/880.2*np.exp(-qpn*z)*b[i]*z**-i 
        
    #Kawano rate
    #rate=1/879.6*(5.252/z - 16.229/z**2 + 18.059/z**3 + 34.181/z**4 + 27.617/z**5)*np.exp(-2.530988*z)
    rate_eval.p__n = rate

'''

In [91]:
n__p_string = '''
a0 = 1
a1 = 0.15735 
a2 = 4.6172
a3 = -0.40520e2 
a4 = 0.13875e3 
a5 = -0.59898e2
a6 = 0.66752e2 
a7 = -0.16705e2 
a8 = 3.8071
a9 = -0.39140 
a10 = 0.023590 
a11 = -0.83696e-4
a12 = -0.42095e-4 
a13 = 0.17675e-5
qnp = 0.33979 

@numba.njit()
def n__p(rate_eval, tf):
    # n --> p
    z=5.92989658*tf.T9i
    #rate from https://arxiv.org/pdf/astro-ph/0408076.pdf appendix C
    rate=0
    a=[a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13]
    for i in range(14):
      rate+=1/880.2*np.exp(-qnp/z)*a[i]*z**-i

    #Kawano rate
    #rate = 1/879.6*(0.565/z - 6.382/z**2 + 11.108/z**3 + 36.492/z**4 + 27.512/z**5)

    rate_eval.n__p = rate

'''

In [92]:
class n__p_Rate(pyna.Rate):
    def __init__(self, reactants=None, products=None,
                 r0=1.0, T0=1.0, nu=0):

        # we'll take the Q value just to be the change in binding energy
        Q = 0
        for n in reactants:
            Q += -n.A * n.nucbind
        for n in products:
            Q += n.A * n.nucbind

        # we set the chapter to custom so the network knows how to deal with it
        self.chapter = "custom"
        self.reverse = None
    
        # call the Rate init to do the remaining initialization
        super().__init__(reactants=reactants, products=products, Q=Q)

        self.r0 = r0
        self.T0 = T0
        self.nu = nu

    def function_string_py(self):
        """return a string containing a python function that computes
        the rate"""
        return n__p_string

    def eval(self, T, rhoY=None):
        return self.r0 * (T / self.T0)**self.nu

In [93]:
n__p = n__p_Rate(reactants=[pyna.Nucleus("n")],
                  products=[pyna.Nucleus("p")],
                  r0=1, T0=1, nu=1)

In [94]:
class p__n_Rate(pyna.Rate):
    def __init__(self, reactants=None, products=None,
                 r0=1.0, T0=1.0, nu=0):

        # we'll take the Q value just to be the change in binding energy
        Q = 0
        for n in reactants:
            Q += -n.A * n.nucbind
        for n in products:
            Q += n.A * n.nucbind

        # we set the chapter to custom so the network knows how to deal with it
        self.chapter = "custom"

        self.reverse = None
    
        # call the Rate init to do the remaining initialization
        super().__init__(reactants=reactants, products=products, Q=Q)

        self.r0 = r0
        self.T0 = T0
        self.nu = nu

    def function_string_py(self):
        """return a string containing a python function that computes
        the rate"""
        return p__n_string

    def eval(self, T, rhoY=None):
        return self.r0 * (T / self.T0)**self.nu

In [95]:
p__n = p__n_Rate(reactants=[pyna.Nucleus("p")],
                  products=[pyna.Nucleus("n")],
                  r0=1, T0=1, nu=1)

In [96]:
p__n.fname='p__n'
n__p.fname='n__p'
#print(p__n.function_string_py())

In [97]:
bbn_network.validate(reaclibrary)

validation: o16 produced in C12 + He4 ⟶ O16 + 𝛾 never consumed.
validation: o16 produced in C13 + He4 ⟶ n + O16 never consumed.
validation: o16 produced in N13 + He4 ⟶ p + O16 never consumed.
validation: o16 produced in N15 + p ⟶ O16 + 𝛾 never consumed.
validation: o16 produced in O15 + n ⟶ O16 + 𝛾 never consumed.


validation: missing Li7 + He4 ⟶ p + Be10 as alternative to Li7 + He4 ⟶ B11 + 𝛾 (Q = -2.56411 MeV).
validation: missing Li8 + He4 ⟶ p + Be11 as alternative to Li8 + He4 ⟶ n + B11 (Q = -4.09311 MeV).
validation: missing Li8 + He4 ⟶ p + Be11 as alternative to Li8 + He4 ⟶ B12 + 𝛾 (Q = -4.09311 MeV).
validation: missing Li8 ⟶ Be8 + e⁻ + 𝜈 as alternative to Li8 ⟶ He4 + He4 + e⁻ + 𝜈 (Q = 16.004 MeV).
validation: missing B8 ⟶ Be8 + e⁺ + 𝜈 as alternative to B8 ⟶ He4 + He4 + e⁺ + 𝜈 (Q = 17.98 MeV).
validation: missing C13 + n ⟶ He4 + Be10 as alternative to C13 + n ⟶ C14 + 𝛾 (Q = -3.83555 MeV).
validation: missing C13 + He4 ⟶ p + N16 as alternative to C13 + He4 ⟶ n + O16 (Q = -7.42111 MeV).
validation: missing C14 ⟶ He4 + Be10 as alternative to C14 ⟶ N14 + e⁻ + 𝜈 (Q = -12.0119 MeV).
validation: missing N12 + He4 ⟶ n + F15 as alternative to N12 + He4 ⟶ p + O15 (Q = -5.085 MeV).
validation: missing N13 + p ⟶ n + O13 as alternative to N13 + p ⟶ O14 + 𝛾 (Q = -18.548 MeV).
validation: missing N13 + He

False

In [98]:
class RatePair:
    """the forward and reverse rates for a single reaction sequence.
    Forward rates are those with Q >= 0.

    :var forward: the forward reaction Rate object
    :var reverse: the reverse reaction Rate object

    """

    def __init__(self, forward=None, reverse=None):
        self.forward = forward
        self.reverse = reverse

    def __repr__(self):
        return f"forward: {self.forward} ; reverse: {self.reverse}"

    def __lt__(self, other):
        if self.forward is not None and other.forward is not None:
            return self.forward < other.forward
        if self.forward is None:
            return False
        return True

    def __eq__(self, other):
        return self.forward == other.forward and self.reverse == other.reverse


In [99]:
bothrates=RatePair(p__n,n__p)

In [100]:
bothrates

forward: p ⟶ n + e⁺ + 𝜈 ; reverse: n ⟶ p + e⁻ + 𝜈

In [101]:
bbn_library += pyna.Library(rates=[p__n,n__p])
bbn_library.remove_rate(bbn_library.get_rate('n__p__weak__wc12'))

In [102]:
bbn_network = pyna.networks.PythonNetwork(libraries=bbn_library)

In [103]:
bbn_network.find_duplicate_links()

[[p + p ⟶ H2 + e⁺ + 𝜈, p + p + e⁻ ⟶ H2 + 𝜈]]

In [104]:
networkname='full_size_net.py'
bbn_network.write_network(networkname)

file=open(networkname, 'a')
file.write('''
#For AoT compilation of the network
def AoT(networkname):
   from numba.pycc import CC

   cc = CC(networkname)
   # Uncomment the following line to print out the compilation steps
   #cc.verbose = True

   #
   @cc.export('nnuc','i4()')
   def nNuc():
      return nnuc

   @cc.export('rhs', 'f8[:](f8, f8[:], f8, f8)')
   def rhsCC(t, Y, rho, T):
      return rhs_eq(t, Y, rho, T, None)

   @cc.export('jacobian', '(f8, f8[:], f8, f8)')
   def jacobian(t, Y, rho, T):
      return jacobian_eq(t, Y, rho, T, None)


   if __name__ == "__main__":
      cc.compile()
''') # Write some text
file.close() # Close the file





In [105]:
bbn_network.validate(reaclibrary)

validation: o16 produced in C12 + He4 ⟶ O16 + 𝛾 never consumed.
validation: o16 produced in C13 + He4 ⟶ n + O16 never consumed.
validation: o16 produced in N13 + He4 ⟶ p + O16 never consumed.
validation: o16 produced in N15 + p ⟶ O16 + 𝛾 never consumed.
validation: o16 produced in O15 + n ⟶ O16 + 𝛾 never consumed.


validation: missing Li7 + He4 ⟶ p + Be10 as alternative to Li7 + He4 ⟶ B11 + 𝛾 (Q = -2.56411 MeV).
validation: missing Li8 + He4 ⟶ p + Be11 as alternative to Li8 + He4 ⟶ n + B11 (Q = -4.09311 MeV).
validation: missing Li8 + He4 ⟶ p + Be11 as alternative to Li8 + He4 ⟶ B12 + 𝛾 (Q = -4.09311 MeV).
validation: missing Li8 ⟶ Be8 + e⁻ + 𝜈 as alternative to Li8 ⟶ He4 + He4 + e⁻ + 𝜈 (Q = 16.004 MeV).
validation: missing B8 ⟶ Be8 + e⁺ + 𝜈 as alternative to B8 ⟶ He4 + He4 + e⁺ + 𝜈 (Q = 17.98 MeV).
validation: missing C13 + n ⟶ He4 + Be10 as alternative to C13 + n ⟶ C14 + 𝛾 (Q = -3.83555 MeV).
validation: missing C13 + He4 ⟶ p + N16 as alternative to C13 + He4 ⟶ n + O16 (Q = -7.42111 MeV).
validation: missing C14 ⟶ He4 + Be10 as alternative to C14 ⟶ N14 + e⁻ + 𝜈 (Q = -12.0119 MeV).
validation: missing N12 + He4 ⟶ n + F15 as alternative to N12 + He4 ⟶ p + O15 (Q = -5.085 MeV).
validation: missing N13 + p ⟶ n + O13 as alternative to N13 + p ⟶ O14 + 𝛾 (Q = -18.548 MeV).
validation: missing N13 + He

False

In [106]:
print(bbn_network.network_overview())

n
  consumed by:
     n + p ⟶ H2 + 𝛾
     H2 + n ⟶ H3 + 𝛾
     He3 + n ⟶ He4 + 𝛾
     Li6 + n ⟶ Li7 + 𝛾
     Li7 + n ⟶ Li8 + 𝛾
     B10 + n ⟶ B11 + 𝛾
     B11 + n ⟶ B12 + 𝛾
     C11 + n ⟶ C12 + 𝛾
     C12 + n ⟶ C13 + 𝛾
     C13 + n ⟶ C14 + 𝛾
     N13 + n ⟶ N14 + 𝛾
     N14 + n ⟶ N15 + 𝛾
     O14 + n ⟶ O15 + 𝛾
     O15 + n ⟶ O16 + 𝛾
     He3 + n ⟶ p + H3
     He3 + n ⟶ H2 + H2
     n + He4 ⟶ H2 + H3
     Li6 + n ⟶ He4 + H3
     Be7 + n ⟶ p + Li7
     Be7 + n ⟶ H2 + Li6
     Be7 + n ⟶ He4 + He4
     Be9 + n ⟶ H2 + Li8
     Be9 + n ⟶ H3 + Li7
     B10 + n ⟶ He4 + Li7
     B11 + n ⟶ H3 + Be9
     B11 + n ⟶ He4 + Li8
     C11 + n ⟶ p + B11
     C12 + n ⟶ p + B12
     C12 + n ⟶ He4 + Be9
     N13 + n ⟶ p + C13
     N13 + n ⟶ He4 + B10
     N14 + n ⟶ p + C14
     N14 + n ⟶ H2 + C13
     N14 + n ⟶ He4 + B11
     N15 + n ⟶ H2 + C14
     N15 + n ⟶ He4 + B12
     O14 + n ⟶ p + N14
     O14 + n ⟶ He4 + C11
     O15 + n ⟶ p + N15
     O15 + n ⟶ He4 + C12
     O16 + n ⟶ He4 + C13
     B8 + n ⟶ p + H

In [107]:
filter = pyna.RateFilter(products=["He4"], exact=False)
He4rates = bbn_library.filter(filter).get_rates()
He4rates

[Li6 ⟶ He4 + H2,
 Li7 ⟶ He4 + H3,
 Li8 ⟶ He4 + He4 + e⁻ + 𝜈,
 Be7 ⟶ He4 + He3,
 B8 ⟶ He4 + He4 + e⁺ + 𝜈,
 B10 ⟶ He4 + Li6,
 B11 ⟶ He4 + Li7,
 B12 ⟶ He4 + Li8,
 C11 ⟶ He4 + Be7,
 O16 ⟶ He4 + C12,
 Li6 ⟶ n + p + He4,
 Be9 ⟶ n + He4 + He4,
 C12 ⟶ He4 + He4 + He4,
 H2 + H2 ⟶ He4 + 𝛾,
 H3 + p ⟶ He4 + 𝛾,
 He3 + n ⟶ He4 + 𝛾,
 He3 + p ⟶ He4 + e⁺ + 𝜈,
 H3 + H2 ⟶ n + He4,
 He3 + H2 ⟶ p + He4,
 He3 + H3 ⟶ H2 + He4,
 Li6 + n ⟶ He4 + H3,
 Li6 + p ⟶ He4 + He3,
 Li7 + p ⟶ He4 + He4,
 Be7 + n ⟶ He4 + He4,
 Be9 + p ⟶ He4 + Li6,
 B10 + n ⟶ He4 + Li7,
 B10 + p ⟶ He4 + Be7,
 B11 + n ⟶ He4 + Li8,
 B12 + p ⟶ He4 + Be9,
 C11 + p ⟶ He4 + B8,
 C12 + n ⟶ He4 + Be9,
 C13 + p ⟶ He4 + B10,
 C14 + p ⟶ He4 + B11,
 N13 + n ⟶ He4 + B10,
 N14 + n ⟶ He4 + B11,
 N14 + p ⟶ He4 + C11,
 N15 + n ⟶ He4 + B12,
 N15 + p ⟶ He4 + C12,
 O14 + n ⟶ He4 + C11,
 O15 + n ⟶ He4 + C12,
 O15 + p ⟶ He4 + N12,
 O16 + n ⟶ He4 + C13,
 O16 + p ⟶ He4 + N13,
 H3 + H3 ⟶ n + n + He4,
 He3 + H3 ⟶ n + p + He4,
 He3 + He3 ⟶ p + p + He4,
 Li7 + H2 ⟶ n

In [108]:
TMeV2T9=11.60451812
Tpeak=0.5256676288178614*TMeV2T9

for rate in He4rates:
    print(rate.fname+'\t'+str(rate.eval(Tpeak)))


li6__he4_d	0.0
li7__he4_t	0.0
li8__he4_he4__weak__wc12	0.8251748299560883
be7__he4_he3	0.0
b8__he4_he4__weak__wc12	0.9001912844167906
b10__he4_li6	0.0
b11__he4_li7	0.0
b12__he4_li8	0.0
c11__he4_be7	0.0
o16__he4_c12	0.0
li6__n_p_he4	0.0
be9__n_he4_he4	0.0
c12__he4_he4_he4	0.0
d_d__he4	0.0
p_t__he4	0.0
n_he3__he4	248.91468501748517
p_he3__he4__weak__bet_pos_	0.0
d_t__n_he4	0.0
d_he3__p_he4	0.0
t_he3__d_he4	0.0
n_li6__he4_t	0.0
p_li6__he4_he3	0.0
p_li7__he4_he4	0.0
n_be7__he4_he4	0.4633554890886751
p_be9__he4_li6	0.0
n_b10__he4_li7	506887680.73465866
p_b10__he4_be7	0.0
n_b11__he4_li8	0.0
p_b12__he4_be9	0.0
p_c11__he4_b8	0.0
n_c12__he4_be9	0.0
p_c13__he4_b10	0.0
p_c14__he4_b11	0.0
n_n13__he4_b10	0.0
n_n14__he4_b11	0.0
p_n14__he4_c11	0.0
n_n15__he4_b12	0.0
p_n15__he4_c12	0.0
n_o14__he4_c11	inf
n_o15__he4_c12	34974437.92704896
p_o15__he4_n12	0.0
n_o16__he4_c13	0.0
p_o16__he4_n13	0.0
t_t__n_n_he4	0.0
t_he3__n_p_he4	0.0
he3_he3__p_p_he4	0.0
d_li7__n_he4_he4	0.0
p_li8__n_he4_he4	0.0
d_be7__p_he

In [109]:
rate

sorted(He4rates,key=lambda rate: rate.eval(Tpeak),reverse=True)

[O14 + n ⟶ He4 + C11,
 B10 + n ⟶ He4 + Li7,
 B8 + n ⟶ p + He4 + He4,
 C11 + n ⟶ He4 + He4 + He4,
 O15 + n ⟶ He4 + C12,
 He3 + n ⟶ He4 + 𝛾,
 B8 ⟶ He4 + He4 + e⁺ + 𝜈,
 Li8 ⟶ He4 + He4 + e⁻ + 𝜈,
 Be7 + n ⟶ He4 + He4,
 Li6 ⟶ He4 + H2,
 Li7 ⟶ He4 + H3,
 Be7 ⟶ He4 + He3,
 B10 ⟶ He4 + Li6,
 B11 ⟶ He4 + Li7,
 B12 ⟶ He4 + Li8,
 C11 ⟶ He4 + Be7,
 O16 ⟶ He4 + C12,
 Li6 ⟶ n + p + He4,
 Be9 ⟶ n + He4 + He4,
 C12 ⟶ He4 + He4 + He4,
 H2 + H2 ⟶ He4 + 𝛾,
 H3 + p ⟶ He4 + 𝛾,
 He3 + p ⟶ He4 + e⁺ + 𝜈,
 H3 + H2 ⟶ n + He4,
 He3 + H2 ⟶ p + He4,
 He3 + H3 ⟶ H2 + He4,
 Li6 + n ⟶ He4 + H3,
 Li6 + p ⟶ He4 + He3,
 Li7 + p ⟶ He4 + He4,
 Be9 + p ⟶ He4 + Li6,
 B10 + p ⟶ He4 + Be7,
 B11 + n ⟶ He4 + Li8,
 B12 + p ⟶ He4 + Be9,
 C11 + p ⟶ He4 + B8,
 C12 + n ⟶ He4 + Be9,
 C13 + p ⟶ He4 + B10,
 C14 + p ⟶ He4 + B11,
 N13 + n ⟶ He4 + B10,
 N14 + n ⟶ He4 + B11,
 N14 + p ⟶ He4 + C11,
 N15 + n ⟶ He4 + B12,
 N15 + p ⟶ He4 + C12,
 O15 + p ⟶ He4 + N12,
 O16 + n ⟶ He4 + C13,
 O16 + p ⟶ He4 + N13,
 H3 + H3 ⟶ n + n + He4,
 He3 + H3 ⟶

In [110]:
#For AoT compilation of the network
'''
from numba.pycc import CC

cc = CC('AoT_net')
# Uncomment the following line to print out the compilation steps
#cc.verbose = True

@cc.export('nnuc','i4()')
def nNuc():
    return nnuc

@cc.export('rhs', 'f8[:](f8, f8[:], f8, f8)')
def rhsCC(t, Y, rho, T):
    return rhs_eq(t, Y, rho, T, None)

@cc.export('jacobian', '(f8, f8[:], f8, f8)')
def jacobian(t, Y, rho, T):
    return jacobian_eq(t, Y, rho, T, None)


if __name__ == "__main__":
    cc.compile()

'''

'\nfrom numba.pycc import CC\n\ncc = CC(\'AoT_net\')\n# Uncomment the following line to print out the compilation steps\n#cc.verbose = True\n\n@cc.export(\'nnuc\',\'i4()\')\ndef nNuc():\n    return nnuc\n\n@cc.export(\'rhs\', \'f8[:](f8, f8[:], f8, f8)\')\ndef rhsCC(t, Y, rho, T):\n    return rhs_eq(t, Y, rho, T, None)\n\n@cc.export(\'jacobian\', \'(f8, f8[:], f8, f8)\')\ndef jacobian(t, Y, rho, T):\n    return jacobian_eq(t, Y, rho, T, None)\n\n\nif __name__ == "__main__":\n    cc.compile()\n\n'